In [53]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from scipy.sparse import load_npz
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

In [7]:
listen_data_small = pd.read_csv('data/listen_data_updated_id', index_col=0)

/Users/alextsai/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [21]:
song_genres = pd.read_csv('data/genre_dummy_features.csv',index_col=0)

In [17]:
data_small_pivot = listen_data_small.pivot(index = 'user_id',columns='item_id',values='listen_count')
data_small_pivot = data_small_pivot.fillna(0)
value_matrix = csr_matrix(data_small_pivot)

In [28]:
item_features_genres = csr_matrix(song_genres.drop(columns='item_id'))

In [29]:
state = np.random.RandomState(seed = 42)
train, test = random_train_test_split(value_matrix,random_state=state)

In [51]:
model_fm = LightFM(loss = 'warp', no_components = 13)
model_fm.fit(train, epochs = 100, item_features = item_features_genres)

In [52]:
aucs = auc_score(model_fm, train, item_features=item_features_genres)

In [53]:
aucs.mean()

0.47604162

In [54]:
test_aucs = auc_score(model_fm, test, train_interactions = train, item_features=item_features_genres)

In [55]:
test_aucs.mean()

0.38775283

In [ ]:
pd.read_csv('data/track_tag_features')

In [3]:
import joblib
import boto3 
from io import BytesIO 

def read_joblib(path):
    ''' 
       Function to load a joblib file from an s3 bucket or local directory.
       Arguments:
       * path: an s3 bucket or local directory path where the file is stored
       Outputs:
       * file: Joblib file loaded
    '''

    # Path is an s3 bucket
    if path[:5] == 's3://':
        s3_bucket, s3_key = path.split('/')[2], path.split('/')[3:]
        s3_key = '/'.join(s3_key)
        with BytesIO() as f:
            boto3.client("s3").download_fileobj(Bucket=s3_bucket, Key=s3_key, Fileobj=f)
            f.seek(0)
            file = joblib.load(f)
    
    # Path is a local directory 
    else:
        with open(path, 'rb') as f:
            file = joblib.load(f)
    
    return file

In [33]:
model = read_joblib('s3://a-t-first-bucket/song_recommender_data/light_fm_model.joblib')

In [52]:
model.get_item_representations()

(array([-0.61656314,  0.        ,  0.        , ...,  0.        ,
         0.04867779, -0.341283  ], dtype=float32),
 array([[-0.00752626,  0.5248099 , -0.3807957 , ...,  0.93645895,
         -0.33269346, -0.4773778 ],
        [-0.00738184, -0.01830336, -0.03805493, ...,  0.04226573,
          0.02180647,  0.01255356],
        [-0.00529231,  0.015324  , -0.01964216, ..., -0.03493134,
          0.04419845, -0.03539109],
        ...,
        [ 0.00464477, -0.02945202,  0.03111615, ..., -0.02301037,
          0.04276323,  0.03659171],
        [-0.05398604, -0.48507246, -0.5462581 , ...,  0.49139315,
          0.06381909,  0.4575556 ],
        [-0.10220285,  0.204729  , -0.13882887, ..., -0.42621264,
          0.4900059 ,  0.48266184]], dtype=float32))

In [11]:
tracks = read_joblib('s3://a-t-first-bucket/song_recommender_data/feature_tags.joblib')

In [12]:
table = pd.read_table('http://millionsongdataset.com/sites/default/files/AdditionalFiles/unique_tracks.txt',sep = '<SEP>', header = None )
table.columns = ['track_id','song_id','artist','title']

/Users/alextsai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [14]:
tracks = tracks['track_id']

In [18]:
track_metadata = pd.merge(tracks, table, left_on = tracks, right_on = 'track_id', how = 'left')

In [20]:
track_metadata = track_metadata.drop(columns = ['track_id_x','track_id_y'])

In [22]:
track_metadata = track_metadata.drop(columns = 'song_id')

In [24]:
track_metadata.to_csv('src/track_metadata.csv')

In [25]:
songs = pd.read_csv('data/track_metadata.csv')

In [29]:
songs['ID Number'] = songs.index

In [30]:
songs

,Unnamed: 0,track_id,artist,title,ID Number
0,0,TRAABJS128F9325C99,Beach House,Auburn and Ivory,0
1,1,TRAACIE128F428495B,Pearl Jam,Not For You,1
2,2,TRAACPH12903CF5F14,Camera Obscura,Double Feature,2
3,3,TRAADKW128E079503A,Tracy Chapman,Fast Car (LP Version),3
4,4,TRAADLN128F14832E9,The Rolling Stones,Angie (1993 Digital Remaster),4
...,...,...,...,...,...
9995,9995,TRZZSQU128F92FCDE8,Lil Wayne,Hit Em Up,9995
9996,9996,TRZZUBQ128F9351D51,Lily Allen,Take What You Take,9996
9997,9997,TRZZUTD12903CADD68,Kid Cudi,Solo Dolo (nightmare),9997
9998,9998,TRZZYMU128E0792400,Chris Cornell,Flutter Girl,9998


In [31]:
feature_tags = read_joblib('s3://a-t-fist-bucket/feature_tags.joblib')

ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

In [32]:
feature_tags = pd.read_csv('s3://a-t-first-bucket/track_tag_features')

PermissionError: Forbidden

In [67]:
actual = np.array(['Positive','Positive','Negative','Positive','Negative'])
predicted = np.array([0.956, 0.742, 0.643, 0.542, 0.119])

In [68]:
eval_ = pd.DataFrame(predicted, columns = ['Model Prediction'])

In [69]:
eval_['Actual'] = actual

In [71]:
eval_

,Model Prediction,Actual
0,0.956,Positive
1,0.742,Positive
2,0.643,Negative
3,0.542,Positive
4,0.119,Negative
